# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [11]:
import numpy as np

def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

arr = np.array(list(square_root_generator(limit)))

print('\n' + str(arr.sum()))


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989

32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [27]:
import dlt

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

# Step 1
pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


info = pipeline.run(people_1(),
										table_name="people_1",
										write_disposition="replace")

print(info)



{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Pipeline dlt_ipykernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/dylanlam/Jupyter Pro

In [28]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


df1 = conn.sql("SELECT * FROM people_1").df()
display(df1)
print("Sum of all ages = " + str(df1['age'].sum()))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ http_download       │
│ people_1            │
│ stream_download     │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707897259.374791,CqHWmjRqcUhvUg,None
1,2,Person_2,27,City_A,1707897259.374791,01F30AUQSSmqNA,None
2,3,Person_3,28,City_A,1707897259.374791,2tZ84VoE8ssJ0g,None
3,4,Person_4,29,City_A,1707897259.374791,1oaWwgdOFdRifA,None
4,5,Person_5,30,City_A,1707897259.374791,eV12DsUu/kqk3Q,None


Sum of all ages = 140


In [29]:
# Step 2
info = pipeline.run(people_2(),
										table_name="people_1",
										write_disposition="append")

print(info)

# Step 3
df1 = conn.sql("SELECT * FROM people_1").df()
display(df1)
print("Sum of all ages = " + str(df1['age'].sum()))

Pipeline dlt_ipykernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/dylanlam/Jupyter Projects/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707897271.8438861 is LOADED and contains no failed jobs


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707897259.374791,CqHWmjRqcUhvUg,None
1,2,Person_2,27,City_A,1707897259.374791,01F30AUQSSmqNA,None
2,3,Person_3,28,City_A,1707897259.374791,2tZ84VoE8ssJ0g,None
3,4,Person_4,29,City_A,1707897259.374791,1oaWwgdOFdRifA,None
4,5,Person_5,30,City_A,1707897259.374791,eV12DsUu/kqk3Q,None
5,3,Person_3,33,City_B,1707897271.8438861,oaTrcHcFRTvk2w,Job_3
6,4,Person_4,34,City_B,1707897271.8438861,VrOjthJQYeG7xQ,Job_4
7,5,Person_5,35,City_B,1707897271.8438861,YI1CikBve2c+JA,Job_5
8,6,Person_6,36,City_B,1707897271.8438861,8FbZ9EMbTxrB3A,Job_6
9,7,Person_7,37,City_B,1707897271.8438861,Kx9yo+vj+J3M/A,Job_7


Sum of all ages = 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
#%%capture
#!pip install dlt[duckdb]

In [31]:
pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


info = pipeline.run(people_1(),
										table_name="people_1",
										write_disposition="replace",
										primary_key="id")

print(info)

info = pipeline.run(people_2(),
										table_name="people_1",
										write_disposition="merge")

print(info)

df1 = conn.sql("SELECT * FROM people_1").df()
display(df1)
print("Sum of all ages = " + str(df1['age'].sum()))

Pipeline dlt_ipykernel_launcher load step completed in 0.22 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/dylanlam/Jupyter Projects/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707897671.913846 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/dylanlam/Jupyter Projects/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707897672.5836318 is LOADED and contains no failed jobs


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707897671.913846,iaXzu42muxNT/w,None
1,2,Person_2,27,City_A,1707897671.913846,f81TyGlem9HfgQ,None
2,5,Person_5,35,City_B,1707897672.5836318,81Td+x68qTV/Lg,Job_5
3,3,Person_3,33,City_B,1707897672.5836318,h8kVUETT6NW8kw,Job_3
4,6,Person_6,36,City_B,1707897672.5836318,Tn9wv0ZXD6fIvQ,Job_6
5,7,Person_7,37,City_B,1707897672.5836318,OGR4OdcNsnqh2Q,Job_7
6,4,Person_4,34,City_B,1707897672.5836318,BkQamVep6RJVsA,Job_4
7,8,Person_8,38,City_B,1707897672.5836318,+Dg1QC37AgeLqA,Job_8


Sum of all ages = 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX